In [1]:
import os
#list the current work dir
os.getcwd()
#change the current work dir
os.chdir('/home/siyuan/Seafile//PhD-Siyuan-Niu/Publications/2022/QAOA_DD/CODE/DD_PE/')

In [2]:
from networkx.generators.random_graphs import erdos_renyi_graph
from src.QAOA.QAOA_approximation import generate_three_regular_graph, generate_graph_matrix, create_qaoa_circ, compute_expectation
from scipy.optimize import minimize
import numpy as np

In [3]:
from qiskit import IBMQ, transpile, Aer
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-france', group='univ-montpellier', project='default')
backend = provider.get_backend('ibmq_toronto')

In [4]:
from src.tools.DD_insertion import pm_DD_sequences
from qiskit.transpiler import PassManager, InstructionDurations

durations = InstructionDurations.from_backend(backend)
pms = pm_DD_sequences(durations)

In [5]:
from src.QAOA.QAOA_approximation import get_ansatz_parm

In [6]:
##Construct 3-regular graphs of different degree
weight_matrices = []
graphs = []
circuits = []

for i in range(4, 13, 2):
    G = generate_three_regular_graph(i)
    res = get_ansatz_parm(G)
    print(res)
    qc = create_qaoa_circ(G, res.x)
    w = generate_graph_matrix(G)
    weight_matrices.append(w)
    graphs.append(G)
    circuits.append(qc)
    print('----------')

     fun: -3.693359375
   maxcv: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 33
  status: 1
 success: True
       x: array([52.12044927, 92.93027933])
----------
     fun: -5.3173828125
   maxcv: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 27
  status: 1
 success: True
       x: array([52.14027902, 93.93254817])
----------
     fun: -6.0341796875
   maxcv: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 33
  status: 1
 success: True
       x: array([50.97493708, 91.75939733])
----------
     fun: -9.2822265625
   maxcv: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 32
  status: 1
 success: True
       x: array([51.43942058, 92.4067444 ])
----------
     fun: -9.5009765625
   maxcv: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 35
  status: 1
 success: True
       x: array([52.1257119 , 92.94228148])
----------


In [7]:
## construct randomized graph of different degree
seed = 200
p = 0.5
for i in range(4, 13, 2):
    G = erdos_renyi_graph(i, p, seed)
    res = get_ansatz_parm(G)
    print(res)
    qc = create_qaoa_circ(G, res.x)
    w = generate_graph_matrix(G)
    weight_matrices.append(w)
    graphs.append(G)
    circuits.append(qc)
    print('----------')

     fun: -2.0634765625
   maxcv: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 26
  status: 1
 success: True
       x: array([51.48692938, 93.00036061])
----------
     fun: -6.861328125
   maxcv: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 29
  status: 1
 success: True
       x: array([52.08505957, 92.89675027])
----------
     fun: -11.4970703125
   maxcv: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 29
  status: 1
 success: True
       x: array([52.07542478, 92.83308516])
----------
     fun: -14.072265625
   maxcv: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 32
  status: 1
 success: True
       x: array([49.79069762, 93.1481553 ])
----------
     fun: -19.521484375
   maxcv: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 35
  status: 1
 success: True
       x: array([52.12625061, 90.93369183])
----------


In [8]:
## add duration of y gates which are used for DD sequences
bconf = backend.configuration()
for i in range(bconf.num_qubits):
    x_duration = durations.get('x', i)
    durations.update(InstructionDurations(
        [('y', i, x_duration)]
        ))

    durations.update(InstructionDurations(
        [('rx', i, x_duration)]
        ))

    durations.update(InstructionDurations(
        [('ry', i, x_duration)]
        ))

In [9]:
# Template optimisation for RZZ gate and SWAP gate

from qiskit.circuit.library.standard_gates.equivalence_library import StandardEquivalenceLibrary as std_eqlib

# Transpiler passes
from qiskit.transpiler.passes import Collect2qBlocks
from qiskit.transpiler.passes import ConsolidateBlocks
from qiskit.transpiler.passes import Optimize1qGatesDecomposition
from qiskit.transpiler.passes.basis import BasisTranslator, UnrollCustomDefinitions
from qiskit.transpiler.passes.scheduling.calibration_creators import RZXCalibrationBuilderNoEcho
from qiskit.transpiler.passes.optimization.echo_rzx_weyl_decomposition import EchoRZXWeylDecomposition

In [10]:
inst_map = backend.defaults().instruction_schedule_map
rzx_basis = ['rzx', 'rz', 'x', 'sx']

pm_pulse_efficient = PassManager(
        [
            # Consolidate consecutive two-qubit operations
            Collect2qBlocks(),
            ConsolidateBlocks(basis_gates=['rz', 'sx', 'x', 'rxx']),
            # Rewrite circuit in terms of Cartan-decomposed echoed RZX gates
            EchoRZXWeylDecomposition(inst_map),
            # Calibrations for RZX gates
            RZXCalibrationBuilderNoEcho(backend),
            # Rewrite in rzx basis
            UnrollCustomDefinitions(std_eqlib, rzx_basis),
            BasisTranslator(std_eqlib, rzx_basis),
            Optimize1qGatesDecomposition(rzx_basis)
        ]
    )


In [11]:
from src.tools.DD_insertion import translate_circuit_to_basis, rzx_gate_recover

In [ ]:
## combine template optimisation and dd sequences

job_ids = []
jobs = []

for circuit in circuits:
    circuit_list = []
    seed = 1
    transpiled_qc = transpile(circuit, backend=backend, optimization_level=3, seed_transpiler=seed)
    circuit_list.append(transpiled_qc)
    for pm in pms:
        qc_transpile = pm.run(transpiled_qc)
        qc_transpile_base = translate_circuit_to_basis(qc_transpile, bconf)
        circuit_list.append(qc_transpile_base)

    qc_pulse_efficient = pm_pulse_efficient.run(transpiled_qc)
    qc_pulse_efficient_transpile = rzx_gate_recover(qc_pulse_efficient)
    circuit_list.append(qc_pulse_efficient_transpile)

    qc_pulse_calibrations = qc_pulse_efficient.calibrations
    rzx_durations = []

    for key, value in qc_pulse_calibrations.items():
        rzx_duration = tuple()
        for i, elm in value.items():
            rzx_durations.append((elm.name, list(i[0]), elm.duration))
    durations.update(InstructionDurations(rzx_durations))

    pms2 = pm_DD_sequences(durations)

    for pm in pms2:
        qc_transpile = pm.run(qc_pulse_efficient)
        qc_transpile_base = translate_circuit_to_basis(qc_transpile, bconf)
        qc_transpile = rzx_gate_recover(qc_transpile_base)
        circuit_list.append(qc_transpile)

    job = backend.run(circuit_list, shots=8192)
    jobs.append(job)
    job_id = job.job_id()
    print(job_id)
    job_ids.append(job_id)

In [ ]:
for job in job_ids:
    print(job)

In [15]:
from src.QAOA.QAOA_approximation import expectation_result, get_operator
from qiskit.algorithms import NumPyMinimumEigensolver
shots = 8192

In [ ]:
name = ['Original_ideal', 'hahn_X_ideal', 'hahn_Y_ideal', 'cp_ideal', 'cpmg_ideal', 'xy4_ideal', 'xy8_ideal', 'xy16_ideal', 'udd_x_ideal', 'udd_y_ideal', 'KDD_ideal',
        'Original_pe', 'hahn_X_pe', 'hahn_Y_pe', 'cp_pe', 'cpmg_pe', 'xy4_pe', 'xy8_pe', 'xy16_pe', 'udd_x_pe', 'udd_y_pe', 'KDD_pe']
npme = NumPyMinimumEigensolver()

job_ids = [
    '62875f91aaa04c0de203f286',
'62875f98ae55246c06464312',
'62875fa1158b8b000894395c',
'62875fab158b8bfb6894395d',
'62875fbe9c97e944f42aa6f3',
'62875fca9c97e961fd2aa6f4',
'62875fd2158b8b6e9e94395e',
'62875fdfe2f2781006fbb404',
'62875ff1aaa04c88fe03f28a',
'628760072cf47d0f97385478',
]

graph_name = ['3-regular-4', '3-regular-6', '3-regular-8', '3-regular-10', '3-regular-12',
              'rand-4-0.5', 'rand-6-0.5', 'rand-8-0.5', 'rand-10-0.5', 'rand-12-0.5',]

graph_ratio_results = {}


results_data = {}
for idx, job_id in enumerate(job_ids):
    job = backend.retrieve_job(job_id)
    counts = job.result().get_counts()
    qubit_op, offset, pauli_list = get_operator(weight_matrices[idx])

    approximation_ratios = {}
    exp_val_classical = npme.compute_minimum_eigenvalue(qubit_op).eigenvalue.real
    print(f'--------{graph_name[idx]}----------------')


    approx_ratios =  []
    for i, count in enumerate(counts):
        exp_val_counts = expectation_result(count, shots, pauli_list)
        # print(f'The expectation value of {name[i]} is {exp_val_counts}')

        approximation_ratio = exp_val_counts / exp_val_classical
        approx_ratios.append(approximation_ratio)
        approximation_ratios[name[i]] = approximation_ratio
        print(f'Approximation ratio of {name[i]} is {approximation_ratio}')
    results_data[graph_name[idx]] = approx_ratios
    graph_ratio_results[graph_name[idx]] = approximation_ratios


In [ ]:
print(results_data)

In [ ]:
graph_name = ['3-regular-4', '3-regular-6', '3-regular-8', '3-regular-10', '3-regular-12',
              'rand-4-0.5', 'rand-6-0.5', 'rand-8-0.5', 'rand-10-0.5', 'rand-12-0.5',]
import numpy as np
import matplotlib.pyplot as plt
baseline = []

# name = ['Baseline', 'cp', 'cpmgl', 'xy4', 'xy8', 'xy16', 'udd_x_', 'udd_y', 'kdd',
#         'PE', 'cp_pe', 'cpmg_pe', 'xy4_pe', 'xy8_pe', 'xy16_pe', 'udd_x_pe', 'udd_y_pe','kdd_pe']

data = []
for name in graph_name:
    result_data = results_data[name]
    baseline.append(result_data[0])
    ratio = [i - result_data[0] for i in result_data[1:]]
    data.append(ratio)


data = np.array(data)
# hahn = data[:, 0]
cp = data[:, 2]
cpmg = data[:, 3]
xy4 = data[:, 4]
xy8 = data[:, 5]
xy16 = data[:, 6]
udd_X = data[:, 7]
udd_Y = data[:, 8]
kdd = data[:, 9]

pe = data[:, 10]
# hahn_to = data[:, 10]
cp_pe = data[:, 13]
cpmg_pe = data[:, 14]
xy4_pe = data[:, 15]
xy8_pe = data[:, 16]
xy16_pe = data[:, 17]
udd_X_pe = data[:, 18]
udd_Y_pe = data[:, 19]
kdd_pe = data[:, 20]

graph_name_x = ['3-reg-4', '3-reg-6', '3-reg-8', '3-reg-10', '3-reg-12',
              'rand-4-0.5', 'rand-6-0.5', 'rand-8-0.5', 'rand-10-0.5', 'rand-12-0.5',]
x_ticks = graph_name_x
X = np.arange(len(x_ticks))

from matplotlib.pyplot import figure
fig = figure(num=None, figsize=(8, 6), dpi=600, facecolor='w', edgecolor='k')
ax = fig.add_axes([0,0,1,1])

# plt.plot(X, hahn, linestyle='--', marker='o', color='g', label='hahn')
plt.scatter(X, cp, s=50, marker='o', color='orange', label='cp')
plt.scatter(X, cpmg, s=50, marker='.', color='b', label='cpmg')
plt.scatter(X, xy4, s=50, marker='^', color='r', label='xy4')
plt.scatter(X, xy8, s=50, marker='X', color='g', label='xy8')
plt.scatter(X, xy16, s=50, marker='h', color='c', label='xy16')
plt.scatter(X, udd_X, s=50, marker='d', color='y', label='udd_x')
plt.scatter(X, udd_Y, s=50, marker='+', color='k', label='udd_y')
plt.scatter(X, kdd, s=50, marker='*', color='m', label='kdd')
# plt.plot(X, pe, linestyle='--', marker='>', color='c', label='pe')
# # plt.plot(X, hahn_to, linestyle='--', marker='s', color='m', label='hahn_to')
# plt.plot(X, cp_pe, linestyle='--', marker='P', color='tab:pink', label='cp_pe')
# plt.plot(X, cpmg_pe, linestyle='--', marker='h', color='tab:gray', label='cpmg_pe')
# plt.plot(X, xy4_pe, linestyle='--', marker='_', color='tab:purple', label='xy4_pe')
# plt.plot(X, xy8_pe, linestyle='--', marker='<', color='tab:cyan', label='xy8_pe')
# plt.plot(X, xy16_pe, linestyle='--', marker='8', color='lightgreen', label='xy16_pe')
# plt.plot(X, udd_X_pe, linestyle='--', marker='|', color='tab:brown', label='udd_x_pe')
# plt.plot(X, udd_Y_pe, linestyle='--', marker='X', color='tab:olive', label='udd_y_pe')
# plt.plot(X, kdd_pe, linestyle='--', marker='D', color='maroon', label='kdd_pe')

plt.axhline(0, color='tab:brown', lw=2)

plt.legend(loc='best', fontsize=14)
# ax.set_title('Relative approximate ratio with only DD sequences on ibmq_toronto', fontsize=18)
ax.set_xticks(X)
ax.set_xticklabels(x_ticks)
ax.set_ylabel('Relative approximation ratio', fontsize=16)

plt.savefig('Relat_AR_toronto_DD_0520_TQE.pdf', bbox_inches='tight', pad_inches=0)

In [ ]:

baseline = []

# name = ['Baseline', 'cp', 'cpmgl', 'xy4', 'xy8', 'xy16', 'udd_x_', 'udd_y', 'kdd',
#         'PE', 'cp_pe', 'cpmg_pe', 'xy4_pe', 'xy8_pe', 'xy16_pe', 'udd_x_pe', 'udd_y_pe','kdd_pe']

data = []
for name in graph_name:
    result_data = results_data[name]
    baseline.append(result_data[0])
    ratio = [i - result_data[0] for i in result_data[1:]]
    data.append(ratio)


data = np.array(data)
# hahn = data[:, 0]
cp = data[:, 2]
cpmg = data[:, 3]
xy4 = data[:, 4]
xy8 = data[:, 5]
xy16 = data[:, 6]
udd_X = data[:, 7]
udd_Y = data[:, 8]
kdd = data[:, 9]

pe = data[:, 10]
# hahn_to = data[:, 10]
cp_pe = data[:, 13]
cpmg_pe = data[:, 14]
xy4_pe = data[:, 15]
xy8_pe = data[:, 16]
xy16_pe = data[:, 17]
udd_X_pe = data[:, 18]
udd_Y_pe = data[:, 19]
kdd_pe = data[:, 20]

graph_name_x = ['3-reg-4', '3-reg-6', '3-reg-8', '3-reg-10', '3-reg-12',
              'rand-4-0.5', 'rand-6-0.5', 'rand-8-0.5', 'rand-10-0.5', 'rand-12-0.5',]
x_ticks = graph_name_x
X = np.arange(len(x_ticks))

from matplotlib.pyplot import figure
fig = figure(num=None, figsize=(8, 6), dpi=600, facecolor='w', edgecolor='k')
ax = fig.add_axes([0,0,1,1])

# plt.plot(X, hahn, linestyle='--', marker='o', color='g', label='hahn')
# plt.plot(X, cp, linestyle='--', marker='x', color='orange', label='cp')
# plt.plot(X, cpmg, linestyle='--', marker='.', color='b', label='cpmg')
# plt.plot(X, xy4, linestyle='--', marker='^', color='r', label='xy4')
# plt.plot(X, xy8, linestyle='--', marker='o', color='g', label='xy8')
# plt.plot(X, xy16, linestyle='--', marker='s', color='m', label='xy16')
# plt.plot(X, udd_X, linestyle='--', marker='d', color='y', label='udd_x')
# plt.plot(X, udd_Y, linestyle='--', marker='+', color='k', label='udd_y')
# plt.plot(X, kdd, linestyle='--', marker='v', color='lightcoral', label='kdd')
plt.scatter(X, pe, s=50, marker='P', color='tab:purple', label='pe')
# plt.plot(X, hahn_to, linestyle='--', marker='s', color='m', label='hahn_to')
plt.scatter(X, cp_pe, s=50, marker='o', color='orange', label='cp_pe')
plt.scatter(X, cpmg_pe, s=50, marker='.', color='b', label='cpmg_pe')
plt.scatter(X, xy4_pe, s=50, marker='^', color='r', label='xy4_pe')
plt.scatter(X, xy8_pe, s=50, marker='X', color='g', label='xy8_pe')
plt.scatter(X, xy16_pe, s=50, marker='h', color='c', label='xy16_pe')
plt.scatter(X, udd_X_pe, s=50, marker='d', color='y', label='udd_x_pe')
plt.scatter(X, udd_Y_pe, s=50, marker='+', color='k', label='udd_y_pe')
plt.scatter(X, kdd_pe, s=50, marker='*', color='m', label='kdd_pe')

plt.axhline(0, color='tab:brown', lw=2)

plt.legend(loc='best', fontsize=14)
# ax.set_title('Relative approximate ratio with DD sequences+pulse efficient on ibmq_toronto', fontsize=18)
ax.set_xticks(X)
ax.set_xticklabels(x_ticks)
ax.set_ylabel('Relative approximation ratio', fontsize=16)

plt.savefig('Relat_AR_toronto_DD_pe_0520_TQE.pdf', bbox_inches='tight', pad_inches=0)

In [ ]:
name = ['Original_ideal', 'hahn_X_ideal', 'hahn_Y_ideal', 'cp_ideal', 'cpmg_ideal', 'xy4_ideal', 'xy8_ideal', 'xy16_ideal', 'udd_x_ideal', 'udd_y_ideal', 'KDD_ideal',
        'Original_pe', 'hahn_X_pe', 'hahn_Y_pe', 'cp_pe', 'cpmg_pe', 'xy4_pe', 'xy8_pe', 'xy16_pe', 'udd_x_pe', 'udd_y_pe', 'KDD_pe']
for i in range(2,21):
    print(name[i+1], data[:,i])
    print('average of improvement:',sum(data[:,i])/len(data[:,i]))
    print('----')

In [ ]:
np.max(data)

In [ ]:
#only DD max value
np.max(data[:,2:10])

In [ ]:
(0.1939697265625-0.09281412760416669)/0.09281412760416669